In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
# Cargar los datos
steam_games = pd.read_csv('C:/Users/delfi/Downloads/delfina local/PI MLOps - STEAM - DELFINA/datasets/steam_games.csv')
df_steam_games = pd.DataFrame(steam_games)

In [14]:
df_steam_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"Strategy, Action, Indie, Casual, Simulation",http://steamcommunity.com/app/761140/reviews/?...,Single-player,4.99,False,761140.0,Kotoshiro
1,"Making Fun, Inc.","Free to Play, Indie, RPG, Strategy",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"Free to Play, Strategy, Indie, RPG, Card Game,...",http://steamcommunity.com/app/643980/reviews/?...,"Single-player, Multi-player, Online Multi-Play...",Free To Play,False,643980.0,Secret Level SRL


In [15]:
# Seleccionar las columnas relevantes
columns_to_use = ['genres', 'tags', 'app_name', 'title', 'id']  # Coloca aquí las columnas que necesitas

# Crear un nuevo DataFrame con las columnas seleccionadas
df_selected_columns = df_steam_games[columns_to_use].copy()

# Muestra aleatoria para reducir el tamaño del DataFrame (opcional, para grandes conjuntos de datos)
df_selected_columns = df_selected_columns.sample(frac=0.3, random_state=42)

# Preprocesamiento y combinación de datos relevantes
df_selected_columns['combined_features'] = df_selected_columns['genres'].fillna('') + ' ' + df_selected_columns['tags'].fillna('') + ' ' + df_selected_columns['app_name'].fillna('')

# Liberar memoria de las variables no utilizadas
del df_steam_games

In [16]:
df_selected_columns.head()

,genres,tags,app_name,title,id,combined_features
4903,Action,Action,HELLDIVERS™ - Specialist Pack,HELLDIVERS™ - Specialist Pack,395237.0,Action Action HELLDIVERS™ - Specialist Pack
12034,"Adventure, Indie, Strategy","Strategy, Adventure, Indie",White Noise 2 - Lilith,White Noise 2 - Lilith,691441.0,"Adventure, Indie, Strategy Strategy, Adventure..."
6188,"Casual, Free to Play, Indie, Simulation","Indie, Casual, Simulation, Free to Play, Clicker",AdVenture Capitalist - Savvy Investor Bundle,AdVenture Capitalist - Savvy Investor Bundle,471650.0,"Casual, Free to Play, Indie, Simulation Indie,..."
6072,"Casual, Simulation","Casual, Simulation",Rocksmith® 2014 – Rise Against - “Help Is On T...,Rocksmith® 2014 – Rise Against - “Help Is On T...,436613.0,"Casual, Simulation Casual, Simulation Rocksmit..."
7456,"Action, Indie","Indie, Action, Bullet Hell",Desperate Times,Desperate Times,513920.0,"Action, Indie Indie, Action, Bullet Hell Despe..."


In [17]:
# Creación de la matriz TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_selected_columns['combined_features'])

# Calcular la similitud del coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [19]:
# Función para obtener recomendaciones
def get_recommendations(id, cosine_sim=cosine_sim, df=df_selected_columns):
    """
    Obtiene recomendaciones de juegos similares al producto ingresado mediante su ID.

    Parámetros:
    product_id (int): ID del producto para el cual se desean obtener recomendaciones.
    cosine_sim (array): Matriz de similitud del coseno entre los juegos.
    df (DataFrame): DataFrame con los datos de juegos y características relevantes.

    Retorna:
    pandas.Series o str: Una serie de títulos de juegos recomendados si se encuentran datos para el ID proporcionado.
                         Si no se encuentran datos para el ID, retorna un mensaje indicando que no se encontraron datos.
    """
    # Crear una serie con los índices y los ID de los productos
    indices = pd.Series(df.index, index=df['id']).drop_duplicates()

    # Verificar si el ID del producto está presente en los índices
    if id not in indices:
        return "No se encontraron datos para este ID de producto."
    
    # Obtener el índice del producto según el ID ingresado
    idx = indices[id]

    # Calcular la similitud entre el producto ingresado y los demás productos
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Top 5 juegos similares
    
    # Obtener los índices de los juegos similares
    game_indices = [i[0] for i in sim_scores]
    
    # Devolver los títulos de los juegos similares encontrados
    return df['title'].iloc[game_indices]

In [20]:
# Obtener recomendaciones para el producto con ID 395237.0
recomendaciones = get_recommendations(395237.0)

# Verificar si hay datos disponibles para el ID proporcionado
if isinstance(recomendaciones, str):
    print(recomendaciones)  # Mensaje indicando que no se encontraron datos
else:
    print(recomendaciones)  # Títulos de juegos recomendados


19186               Block King
16235             BALLOON Saga
22457       AXYOS - Soundtrack
32091    PLAYERUNKN4WN: Zombie
6011               Hero Battle
Name: title, dtype: object
